In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Dependencies

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
# !apt install xvfb
!pip install einops
!pip install pyvirtualdisplay
!sudo apt-get install xvfb
!pip install pyglet==1.5.1 gymnasium[atari]
!pip install ale-py
!pip install gymnasium[accept-rom-license]
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit4/requirements-unit4.txt

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0
                          , size=(1400, 900))
virtual_display.start()

In [ ]:
import numpy as np
import warnings
from collections import namedtuple
import random
from itertools import count
warnings.filterwarnings("ignore")

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from einops.layers.torch import Rearrange, Reduce
from torch.distributions import Categorical
from torchvision.transforms.v2 import Resize, Grayscale

# Gymnasium
import gymnasium as gym
import gym_pygame

# Hugging Face Hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.
import imageio

import os
import pyvirtualdisplay
from datetime import datetime
import base64
import io
from IPython import display as ipythondisplay

In [ ]:
"""basic wrappers, useful for reinforcement learning on gym envs"""
# Mostly copy-pasted from https://github.com/openai/baselines/blob/master/baselines/common/atari_wrappers.py
import cv2
from gym import spaces
import gym
from collections import deque
import os
os.environ.setdefault('PATH', '')
cv2.ocl.setUseOpenCL(False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# DQN Algorithm with Wrapper

## Functions

In [ ]:
re_shape = 96
embd = 128
p_shape = 8
num_p = (re_shape//p_shape)**2
fac = 3
num_layer = 2
frame_to_stack = 4

In [ ]:
def nrs(x, dim = -1):
  return vec_norm(x.relu(), dim = dim)**2

def vec_norm(x, dim = -1, epsilon = 1e-10):
  return torch.div(x, torch.linalg.vector_norm(x, dim = dim, keepdim = True) + epsilon)

class att(nn.Module):
  def __init__(self, head = 24):
    super().__init__()
    self.expand = nn.Sequential(nn.LayerNorm(embd), nn.Linear(embd, int(head) * 3, bias = False))
    self.scale = torch.sqrt(torch.tensor(head, dtype = torch.float32))
    self.proj = nn.Linear(int(head), fac * embd)

  def forward(self, x):
    q, k, v = F.gelu(self.expand(x)).chunk(3, -1)
    return F.gelu(self.proj(torch.einsum('...nm,...mf->...nf', nrs(torch.einsum('...nc,...mc->...nm', q, k)/self.scale, dim = -1), v)))

class spatial_proj(nn.Module):
  def __init__(self, p = num_p, c = num_p):
    super(spatial_proj, self).__init__()
    self.proj = nn.Conv1d(p, p, 1, groups = p//c)
    nn.init.constant_(self.proj.weight,0)
    nn.init.constant_(self.proj.bias,1)

  def forward(self, x):
    return self.proj(x)

class gMLP(nn.Module):
  def __init__(self, ):
    super().__init__()
    self.norm = nn.LayerNorm(embd*fac)
    self.expand = nn.Sequential(nn.LayerNorm(embd), nn.Linear(embd, embd*fac*2, bias = False))
    self.proj = spatial_proj()
    self.proj1 = spatial_proj(c = 2)
    self.cp = nn.Linear(embd*fac, embd)
    self.res = att()

  def forward(self, x):
    u, v = F.gelu(self.expand(x)).chunk(2, -1)
    v = self.norm(v)
    return F.gelu(self.cp(u * (self.proj(v) + self.proj1(v) + self.res(x)))) + x

class nnModel(nn.Module):
  def __init__(self, env, num_layer = num_layer):
    super().__init__()
    self.patch = nn.Sequential(nn.BatchNorm2d(frame_to_stack), nn.Conv2d(frame_to_stack, embd, p_shape, p_shape), Rearrange('b n h w-> b (h w) n'))
    self.model = nn.Sequential(*[gMLP() for _ in range(num_layer)])
    self.proj = nn.Sequential(nn.Linear(embd*num_p, embd), nn.Linear(embd, env.action_space.n))

  def forward(self, x):
    #return self.proj(F.relu(self.model(self.patch(x)).mean(dim = 1)))
    return self.proj(F.relu(nn.Flatten()(self.model(self.patch(x)))))

def prepro(x):
  return Resize((re_shape, re_shape))(Grayscale()(Rearrange('b s h w c -> b s c h w')(x)))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

class DQN(nn.Module):
  def __init__(self, env):
    super(DQN, self).__init__()
    self.layer = nnModel(env)
    nn.Sequential(# scale along color channel
    #nn.LayerNorm(3),
# shape -> (b, c, h, w) to fit pytorch cnn

    #Rearrange('b h w c -> b c h w'),
    nn.BatchNorm2d(frame_to_stack),
# conv on h, w
    nn.Conv2d(frame_to_stack, 32, 8, 4,),
# reduce h, w by half
    #nn.MaxPool2d((2,2)),
# activate
    nn.GELU(),
# conv on h, w
    nn.Conv2d(32, 64, 4, 2,),
# reduce h, w by half
    nn.MaxPool2d((2,2)),
# activate
    nn.GELU(),
# conv on h, w
    nn.Conv2d(64, 64, 3, 1,),
# reduce h, w by half
    nn.MaxPool2d((2,2)),
# activate
    nn.GELU(),
# compress the h, w, c into 1D vector
    nn.Flatten(),
# vanilla mlp and project to action space
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, env.action_space.n),
    )

  def forward(self, x):
# Input shape (b, 210, 160, 3)
# shape: (batch, height, width, channel)
    return self.layer(x)

class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.integers(1, self.noop_max + 1)  # pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)


class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)


class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condition for a few frames
            # so it's important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs


class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2:
                self._obs_buffer[0] = obs
            if i == self._skip - 1:
                self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)


class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)


class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env, width=84, height=84, grayscale=True):
        """Warp frames to 84x84 as done in the Nature paper and later work."""
        gym.ObservationWrapper.__init__(self, env)
        self.width = width
        self.height = height
        self.grayscale = grayscale
        if self.grayscale:
            self.observation_space = spaces.Box(low=0, high=255,
                                                shape=(self.height, self.width, 1), dtype=np.uint8)
        else:
            self.observation_space = spaces.Box(low=0, high=255,
                                                shape=(self.height, self.width, 3), dtype=np.uint8)

    def observation(self, frame):
        if self.grayscale:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        if self.grayscale:
            frame = np.expand_dims(frame, -1)
        return frame


class LazyFrames(object):
    r"""Ensures common frames are only stored once to optimize memory use.
    To further reduce the memory use, it is optionally to turn on lz4 to
    compress the observations.
    .. note::
        This object should only be converted to numpy array just before forward pass.
    """

    def __init__(self, frames, lz4_compress=False):
        if lz4_compress:
            from lz4.block import compress
            self.shape = frames[0].shape
            self.dtype = frames[0].dtype
            frames = [compress(frame) for frame in frames]
        self._frames = frames
        self.lz4_compress = lz4_compress

    def __array__(self, dtype=None):
        if self.lz4_compress:
            from lz4.block import decompress
            frames = [np.frombuffer(decompress(frame), dtype=self.dtype).reshape(self.shape) for frame in self._frames]
        else:
            frames = self._frames
        out = np.stack(frames, axis=0)
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self.__array__())

    def __getitem__(self, i):
        return self.__array__()[i]


class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(
            shp[:-1] + (shp[-1] * k,)), dtype=env.observation_space.dtype)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))


def wrapper(env, terminal_on_life_loss=True, noop_max=20, frame_skip=2,
            num_stack=frame_to_stack, screen_size=(re_shape, re_shape), grayscale=True, clip_reward=False):
    """Apply a common set of wrappers for Atari games."""
    if terminal_on_life_loss:
        env = EpisodicLifeEnv(env)
    if noop_max:
        env = NoopResetEnv(env, noop_max=10)
    if frame_skip:
        env = MaxAndSkipEnv(env, skip=frame_skip)
    try:
        if 'FIRE' in env.unwrapped.get_action_meanings():
            env = FireResetEnv(env)
    except Exception:
        pass
    if screen_size or grayscale:
        env = WarpFrame(env, *(screen_size + (grayscale,)))
    if num_stack:
        env = FrameStack(env, num_stack)
    if clip_reward:
        env = ClipRewardEnv(env)
    return env

if not os.environ.get('DISPLAY'):
    pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

def embed_video(video_path, width=640):
    content = io.open(video_path, 'rb').read()
    b64 = base64.b64encode(content)
    video_tag = '''

    '''.format(b64.decode(), width)
    ipythondisplay.display(ipythondisplay.HTML(video_tag))

class VideoRecorder:
    def __init__(self, filename, fps=30):
        self.filename = filename
        self.writer = imageio.get_writer(filename, fps=fps)

    def record_frame(self, env):
        frame = env.render()
        self.writer.append_data(frame)

    def close(self, *args, **kwargs):
        self.writer.close(*args, **kwargs)

    def play(self):
        self.close()
        embed_video(self.filename)

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        self.play()

def select_action(state, test = False):
  global moving_eps
  sample = random.random()
  if test:
    eps = 0
  else:
    eps = moving_eps
    if moving_eps > hyperparameters["end_epsilon"]: moving_eps = moving_eps * hyperparameters["eps_decay"]
  if sample > eps:
    with torch.no_grad():
#argmax expected return in actions
      return policy_net(state).max(1).indices.view(1, 1)
#random action
  else:
    return torch.tensor([[env.action_space.sample()]], device = device, dtype=torch.long)

def optimize():
    if len(memory) < batch_size:
        return
    transitions = memory.sample(batch_size)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([torch.tensor(s, dtype = torch.float32, device = device) for s in batch.next_state
                                               if s is not None])
    #state_batch = torch.cat(tuple(torch.tensor(batch.state, dtype = torch.float32, device = device)))
    #action_batch = torch.cat(tuple(torch.tensor(batch.action, dtype = torch.float32, device = device)))
    #reward_batch = torch.cat(tuple(torch.tensor(batch.reward, dtype = torch.float32, device = device)))
    state_batch = torch.tensor(batch.state, dtype = torch.float32, device = device).squeeze(-1)
    action_batch = torch.tensor(batch.action, dtype = torch.int64, device = device).squeeze()
    reward_batch = torch.tensor(batch.reward, dtype = torch.float32, device = device).squeeze(-1)
    #print(state_batch.shape)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch.reshape(-1,1))

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(batch_size, device=device)
    #print(non_final_next_states.shape)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states.squeeze(-1).reshape(-1, frame_to_stack, re_shape, re_shape)).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * hyperparameters["gamma"]) + reward_batch

    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 10)
    optimizer.step()
    global global_step
    global_step += 1

def Re(x):
  return Rearrange('b h w c -> b c h w')(x)

## Initializer

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))
# Other Environment
# Seaquest-v4
# Pong-v4
# Breakout-v4
# Qbert-v4
# Enduro-v4
# BeamRider-v4
# SpaceInvaders-v4

# Create the env
env_id = 'Seaquest-v4'
env = gym.make(gym.spec(env_id), render_mode='rgb_array')
env = wrapper(env)
env.reset()

# Create the evaluation env
eval_env = gym.make(env_id)

# Get the state space and action space
s_size = env.observation_space.shape[0]
a_size = env.action_space.n

print(s_size)
print(a_size)

hyperparameters = {
    "h_size": 16,
    "n_training_episodes": 1000000,
    "n_evaluation_episodes": 1000,
    "max_t": 1000,
    "gamma": 0.95,
    "start_epsilon": 0.2,
    "end_epsilon":0.05,
    "eps_decay": 1e-4,
    "lr": 5e-3,
    "update_interval":50,
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
  }

criterion = nn.MSELoss()
batch_size = 32
num_episodes = 2
#stepsize = 1e-1

policy_net = DQN(env).to(device)
target_net = DQN(env).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.RAdam(policy_net.parameters(), lr = hyperparameters["lr"], weight_decay = 1e-3)
memory = ReplayMemory(100000)

moving_eps = hyperparameters["start_epsilon"]
global_step = 0

print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
#print("Sample observation", env.observation_space.sample()) # Get a random observation
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action

96
18
_____OBSERVATION SPACE_____ 

The State Space is:  96

 _____ACTION SPACE_____ 

The Action Space is:  18
Action Space Sample 7


## Testing

In [ ]:
# Run the test
counter = 0

for i_episode in range(10):
    print(counter, end=' ')
    counter += 1
    # reset the env every episode
    state = env.reset()
    for t in count():
        action = select_action(torch.tensor(state, dtype = torch.float32, device = device).squeeze(-1).unsqueeze(0))
        observation, reward, done, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)

        if done:
          next_state = None #LazyFrames(torch.zeros((1,frame_to_stack,reshape,reshape)))
        else:
          next_state = observation

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize()

        if (global_step + 1 + hyperparameters["update_interval"])%hyperparameters["update_interval"] == 0:
          target_net.load_state_dict(policy_net.state_dict())
        if done:
            break

def random_filename_experiment_design(iter, game, experiment_design):
    return f'/content/drive/MyDrive/SDSC4001_Project/VideoRecording/SeaQuestWrapper/ExperimentDesign{game}{experiment_design}_{iter}_Wrapper_CNN.mp4'

rwd_list = []
game = env_id
experiment_no = 2 # PLEASE CHANGE, IF NO CHANGE VIDEO GONE
print('Complete')
# Test them to 10 video recording real-time game experience
for i in range(10):
    rwd = 0
    with VideoRecorder(random_filename_experiment_design(i, game, experiment_no)) as rec:
        state = env.reset()
        while True:
            rec.record_frame(env)
            action = select_action(torch.tensor(state, dtype = torch.float32, device = device).squeeze(-1).unsqueeze(0))
            #obs, reward, terminated, truncated , info = env.step(action)
            obs, reward, done , info = env.step(action)

            state = obs#torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)
            rwd += reward
            #done = terminated or truncated
            if done:
                break
    print(rwd)
    rwd_list.append(rwd)
print(rwd_list, np.mean(rwd_list))

0 1 2 3 4 5 6 7 8 9 Complete


0.0


0.0


0.0


0.0


...

0.0


0.0


0.0


0.0


0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0.0


In [ ]:
# Run the test
counter = 0

for i_episode in range(10):
    print(counter, end=' ')
    counter += 1
    # reset the env every episode
    state = env.reset()
    for t in count():
        action = select_action(torch.tensor(state, dtype = torch.float32, device = device).squeeze(-1).unsqueeze(0))
        observation, reward, done, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)

        if done:
          next_state = None #LazyFrames(torch.zeros((1,frame_to_stack,reshape,reshape)))
        else:
          next_state = observation

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize()

        if (global_step + 1 + hyperparameters["update_interval"])%hyperparameters["update_interval"] == 0:
          target_net.load_state_dict(policy_net.state_dict())
        if done:
            break

def random_filename_experiment_design(iter, game, experiment_design):
    return f'/content/drive/MyDrive/SDSC4001_Project/VideoRecording/SpaceInvadersWrapper/ExperimentDesign{game}{experiment_design}_{iter}_Wrapper_CNN.mp4'

rwd_list = []
game = env_id
experiment_no = 3 # PLEASE CHANGE, IF NO CHANGE VIDEO GONE
print('Complete')
# Test them to 10 video recording real-time game experience
for i in range(10):
    rwd = 0
    with VideoRecorder(random_filename_experiment_design(i, game, experiment_no)) as rec:
        state = env.reset()
        while True:
            rec.record_frame(env)
            action = select_action(torch.tensor(state, dtype = torch.float32, device = device).squeeze(-1).unsqueeze(0))
            #obs, reward, terminated, truncated , info = env.step(action)
            obs, reward, done , info = env.step(action)

            state = obs#torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)
            rwd += reward
            #done = terminated or truncated
            if done:
                break
    print(rwd)
    rwd_list.append(rwd)
print(rwd_list, np.mean(rwd_list))

0 1 2 3 4 

Complete


145.0


45.0


30.0


...

70.0


30.0


40.0


25.0


30.0


In [ ]:
def random_filename_experiment_design(iter, game, experiment_design):
    return f'/content/drive/MyDrive/SDSC4001_Project/VideoRecording/SpaceInvadersWrapper/ExperimentDesign{game}{experiment_design}_{iter}_Wrapper.mp4'

rwd_list = []
game = env_id
experiment_no = 4 # PLEASE CHANGE, IF NO CHANGE VIDEO GONE
print('Complete')
# Test them to 10 video recording real-time game experience
for i in range(10):
    rwd = 0
    with VideoRecorder(random_filename_experiment_design(i, game, experiment_no)) as rec:
        state = env.reset()
        while True:
            rec.record_frame(env)
            action = select_action(torch.tensor(state, dtype = torch.float32, device = device).squeeze(-1).unsqueeze(0))
            #obs, reward, terminated, truncated , info = env.step(action)
            obs, reward, done , info = env.step(action)

            state = obs#torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)
            rwd += reward
            # done = terminated or truncated
            if done:
                break
    print(rwd)
    rwd_list.append(rwd)
print(rwd_list, np.mean(rwd_list))

Complete


-7.0


-21.0


-21.0


-21.0


-21.0


-21.0


-21.0


-21.0


KeyboardInterrupt: ignored

In [ ]:
from google.colab import runtime
runtime.unassign()